# Import the libararies and packages

In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Softmax
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json

# Download and Prepare the data

In [2]:
def load_data():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0

    X_train = np.expand_dims(X_train, axis=3)
    X_test = np.expand_dims(X_test, axis=3)

    label_binarizer = LabelBinarizer()
    y_train = label_binarizer.fit_transform(y_train)
    y_test = label_binarizer.fit_transform(y_test)

    return X_train, y_train, X_test, y_test


# Build a network

In [3]:
def build_network():
    input_layer = Input(shape = (28, 28, 1))
    convolution_1 = Conv2D(kernel_size=(2,2), 
                           padding="same", 
                           strides=(2,2), 
                           filters=32)(input_layer)
    activation_1 = ReLU()(convolution_1)
    batch_normalization_1 = BatchNormalization()(activation_1)
    pooling_1 = MaxPooling2D(pool_size=(2,2), 
                             strides=(1,1),
                             padding='same')(batch_normalization_1)
    dropout = Dropout(rate=0.5)(pooling_1)
    flatten = Flatten()(dropout)
    dense_1 = Dense(units=128)(flatten)
    activation_2 = ReLU()(dense_1)
    dense_2 = Dense(units=10)(activation_2)
    output = Softmax()(dense_2)

    network = Model(inputs= input_layer, outputs=output, name='my_model')

    return network
    



# Evaluate a network

In [4]:
def evaluate(model, X_test, y_test):
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f'Accuracy: {accuracy}')

# Prepare the data

In [5]:
X_train, y_train, X_test, y_test = load_data()
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.8)

In [6]:
model = build_network()

# Compile and train the model

In [7]:
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
            metrics=['accuracy'])

model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          epochs=50,
          batch_size=1024,
          verbose=0)

# Save the model

In [8]:
model.save('model_and_weights.hdf5')
loaded_model = load_model('model_and_weights.hdf5')
evaluate(loaded_model, X_test, y_test)

Accuracy: 0.9828000068664551
